# Activities extraction

Questo notebook crea csv per ogni activity tramite chunks di testo.
Ogni csv è specifico di 1 attività.

In [1]:
import pandas as pd
import re
import ast

df = pd.read_csv('dataset/chambers.csv')

# Read the string representation of the list and convert it to a list
df['buffer_camera'] = df['buffer_camera'].apply(ast.literal_eval)
df['buffer_senato'] = df['buffer_senato'].apply(ast.literal_eval)

df.head()

,case_id,buffer_camera,buffer_senato
0,25G00016,[Camera dei deputati (atto numero 400): Presen...,[Senato della Repubblica (atto numero 1275): A...
1,25G00014,[Camera dei deputati (atto numero 2206): Asseg...,[Senato della Repubblica (atto numero 1335): P...
2,25G00011,[Camera dei deputati (atto numero 1835): Prese...,[Senato della Repubblica (atto numero 1239): A...
3,25G00009,[Camera dei deputati (atto numero 2196): Asseg...,[Senato della Repubblica (atto numero 1315): P...
4,25G00008,[Camera dei deputati (atto numero 2031): Asseg...,[Senato della Repubblica (atto numero 1128): P...


In [2]:
print(type(df['buffer_camera']))
print(type(df['buffer_senato']))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


Il codice seguente estrae le attività

In [3]:
buffer_cols = ['buffer_camera', 'buffer_senato']

regex_patterns = {
        "presentazione": r"presentato da(?:ll'|l|gli|\s)+([^,\.]+), in data (\d{1,2} \w+ \d{4})",
        "assegnazione": r"assegnato a(?:lla|lle|l) ([^,]+),.*? il (\d{1,2} \w+ \d{4})",
        "esame_commissione": r"esaminato (?:dalla|dalle|dalla|dai|dalle) ([^,]+),.*? il (\d{1,2} \w+ \d{4})",
        "esame_aula": r"esaminato in aula.*? il (\d{1,2} \w+ \d{4})",
        "approvazione": r"approvato.*? il (\d{1,2} \w+ \d{4})"
    }

In [14]:
# Funzione per convertire le date in formato standard
def convert_italian_date(date_str: str) -> str:
    """
    Convert an Italian date string (e.g., "21 maggio 2019") into the format YYYY-MM-DD.
    
    Parameters:
        date_str (str): Date string in the format "DD mese YYYY".
        
    Returns:
        str: Date in YYYY-MM-DD format.
    """
    month_map = {
        "gennaio": "01", "febbraio": "02", "marzo": "03", "aprile": "04",
        "maggio": "05", "giugno": "06", "luglio": "07", "agosto": "08",
        "settembre": "09", "ottobre": "10", "novembre": "11", "dicembre": "12"
    }

    try:
        # Split the date string
        day, month, year = date_str.split()
        month_number = month_map[month.lower()]  # Convert month name to number

        # Format into YYYY-MM-DD
        formatted_date = pd.to_datetime(f"{year}-{month_number}-{day}").strftime("%Y-%m-%d")

        return formatted_date
    except Exception as e:
        raise ValueError(f"Invalid date format: {date_str}. Error: {e}")


# Funzione per estrarre attività, data ed entità responsabile
def extract_events(text, regex_patterns, case_id, chamber):
    events = []
    
    for activity, pattern in regex_patterns.items():
        matches = re.findall(pattern, text, re.IGNORECASE)
        for match in matches:
            if isinstance(match, tuple):
                resource, date = match
            else:
                resource, date = "", match
            
            events.append({
                "case_id": case_id,
                "activity": activity,
                "chamber": chamber,
                #"time": pd.to_datetime(date, format="%d %B %Y", errors='coerce').strftime('%Y-%m-%d'),
                "time": convert_italian_date(date),
                "resource": resource.strip(),
                "chunk": text
            })
    
    return events

In [15]:
# Estrae le attività, le date e le entità responsabili
event_log = [] # lista finale degli eventi

for _, row in df.iterrows():
    case_id = row['case_id']
    
    for buffer_col in buffer_cols:
        chamber = "camera" if buffer_col == "buffer_camera" else "senato"
        
        for text in row[buffer_col]:
            print(case_id,text)
            event_log.extend(extract_events(text, regex_patterns, case_id, chamber))

# Creazione DataFrame finale
df_event_log = pd.DataFrame(event_log)
df_event_log.head()
df_event_log.to_csv("dataset/event_log.csv", index=False)
print("Event log salvato con successo!")

25G00016 Camera dei deputati (atto numero 400): Presentato dall'Onorevole Marco Simiani (PD-IDP) e altri, in data 19 ottobre 2022.
25G00016 Assegnato alla 8 Commissione (Ambiente, territorio e lavori pubblici), in sede referente, il 23 novembre 2022, con i pareri delle commissioni 1 (Affari costituzionali, della Presidenza del Consiglio e Interni), 2 (Giustizia), 5 (Bilancio, tesoro e programmazione), 11 (Lavoro pubblico e privato) e per le Questioni regionali.
25G00016 Esaminato dalla Commissione 8 (Ambiente, territorio e lavori pubblici), in sede referente, il 15, il 21 e il 28 novembre 2023; il 17 gennaio 2024; il 7 febbraio 2024; il 23 aprile 2024; il 5 agosto 2024; il 2 ottobre 2024.
25G00016 Esaminato in Aula il 16 ottobre 2024 e trasferito all'8 Commissione (Ambiente, territorio e lavori pubblici), in sede legislativa, il 17 ottobre 2024.
25G00016 Nuovamente assegnato alla 8 Commissione (Ambiente, territorio e lavori pubblici), in sede legislativa, il 17 ottobre 2024, con i pare